In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep
from google.oauth2 import service_account
from google.cloud import storage 
from datetime import datetime, timedelta
from os import listdir
from os.path import isfile, join
import pandas as pd
import os
import json
import re
import requests

In [4]:
#protocolos
path_protocolos="/home/erison/Desktop/lima/nike/projeto_crawler_zendesk_correios/downloads/netshoes/protocolos/"
path_download_protocolos = path_protocolos
date = datetime.now().strftime('%d%m%Y')

In [7]:
#Crawler
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : path_download_protocolos, "profile.default_content_setting_values.automatic_downloads": 1}
chromeOptions.add_experimental_option("prefs",prefs)
#chromeOptions.add_argument("start-maximized")
browser = webdriver.Chrome(options=chromeOptions, executable_path="/home/erison/Desktop/lima/nike/projeto_crawler_zendesk_correios/chromedriver")
sleep(3)
browser.get("https://marketplace.netshoes.com.br/#/sac/atendimentos-novo")

####LOGIN
sleep(3)
print('CRAWLER PROTOCOLOS - login')
login = WebDriverWait(browser, 2).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div/div/div[2]/div/form/div[1]/div/input')))
sleep(1)
login.send_keys("fribeiro@limaconsulting.com")
sleep(1)
password = browser.find_element(By.NAME, 'senha')
sleep(1)
password.send_keys("Lima@2022")
sleep(2)
submit = browser.find_element('xpath',"/html/body/div/div/div/div/div[2]/div/form/button/span[2]")
sleep(1)
submit.click()
print('CRAWLER PROTOCOLOS - login ok')
sleep(5)


df_protocolo = pd.read_excel('/home/erison/Desktop/lima/nike/projeto_crawler_zendesk_correios/downloads/netshoes/protocolos/teste.xlsx', engine="openpyxl")


print('CRAWLER PROTOCOLOS - Iniciando crawler chat')
lista_cpf = []
lista_chat = []
lista_prio = []  
lista_prod = []
dados_comprador = []
dados_entrega = []
item_troca = []


index = 0
for prot in df_protocolo['Protocolo']:  
        while True:
            try:
                browser.get(f"https://marketplace.netshoes.com.br/#/sac/atendimentos-novo/{prot}")
                sleep(1)
                prioridade=WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#top > div.main > div > div.main-content.ng-scope > section > div.ns-card.ns-card--nohover.ns-top-background-doc.m-b-20 > div.ns-order-info.row.p-l-20.p-r-20.m-t-20 > div.col-xs-5.col-sm-3.col-md-2 > div > p')))
                if prioridade.text != '':  
                    priori = prioridade.text.replace("Prioridade:", "")
                    lista_prio.append(priori)
                    
                    chat = browser.find_element(By.CSS_SELECTOR, '#top > div.main > div > div.main-content.ng-scope > section > div.ns-card.no-p.overflow-no.ns-card--nohover.m-b-20 > div > div.ns-chat__body.ns-chat-js')
                    lista_chat.append(chat.text)

                    cpf_crawler = browser.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/section/div[3]/div[2]/div[1]/p[2]')
                    cpf = cpf_crawler.text.replace('CPF/CNPJ: ', '').replace('.', '').replace('-', '')
                    lista_cpf.append(cpf)
                    sleep(5)
                    
                    
                    detalhes = browser.find_element(By.CSS_SELECTOR, '#top > div.main > div > div.main-content.ng-scope > section > div:nth-child(7) > div > div > table > tbody > tr:nth-child(1) > td:nth-child(15) > a')
                    detalhes.click()
                    sleep(3)
                    
                    
                    ped =  browser.find_element(By.CSS_SELECTOR, '#top > div.main > div > div.main-content.ng-scope > section > div:nth-child(6) > form > table > tbody > tr')
                    ped.click()
                    sleep(2)
                    
                    d_com = browser.find_element(By.CSS_SELECTOR, '#top > div.main > div > div.main-content.ng-scope > section > div:nth-child(6) > form > table > tbody > tr:nth-child(2) > td > div > div > div.row.row-padding.p-5.rel > div:nth-child(1)')
                    dados_comprador.append(d_com.text)
                    sleep(1)
                    
                    d_ent =  browser.find_element(By.CSS_SELECTOR, '#top > div.main > div > div.main-content.ng-scope > section > div:nth-child(6) > form > table > tbody > tr:nth-child(2) > td > div > div > div.row.row-padding.p-5.rel > div:nth-child(2)')
                    dados_entrega.append(d_ent.text)
                    sleep(1)
                    
                    i_tro =  browser.find_element(By.CSS_SELECTOR, '#trocaDevolucao > div:nth-child(2) > div > div > div > table > tbody > tr')
                    item_troca.append(i_tro.text)
                    sleep(1)
                    
                    index += 1
                    print(f'{index} - protocolo: {prot} - {priori}')
                    sleep(2)
                    break                                 

                else:
                    print('Valores vazio - Recarregando pagina')
                    browser.refresh()
                    sleep(2)
                    
            except:
                print('Erro no try - Recarregando pagina')              
                try:   
                    ####LOGIN
                    sleep(5)
                    print('CRAWLER PROTOCOLOS - login')
                    login = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div/div/div[2]/div/form/div[1]/div/input')))
                    sleep(1)
                    login.send_keys("fribeiro@limaconsulting.com")
                    sleep(1)
                    password = browser.find_element(By.NAME, 'senha')
                    sleep(1)
                    password.send_keys("Lima@2022")
                    sleep(2)
                    submit = browser.find_element('xpath',"/html/body/div/div/div/div/div[2]/div/form/button/span[2]")
                    sleep(1)
                    submit.click()
                    print('CRAWLER PROTOCOLOS - login ok')
                    sleep(5)

                    browser.get(f"https://marketplace.netshoes.com.br/#/sac/atendimentos-novo/{prot}")
                    sleep(5)
                    prioridade=WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#top > div.main > div > div.main-content.ng-scope > section > div.ns-card.ns-card--nohover.ns-top-background-doc.m-b-20 > div.ns-order-info.row.p-l-20.p-r-20.m-t-20 > div.col-xs-5.col-sm-3.col-md-2 > div > p')))
                    sleep(5)
                    
                except:
                    print('Recarregando pagina')
                    browser.refresh()
                    sleep(5)

print('CRAWLER PROTOCOLOS - Finalizando crawler chat')
# browser.quit()


/tmp/ipykernel_2354457/1722892525.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=chromeOptions, executable_path="/home/erison/Desktop/lima/nike/projeto_crawler_zendesk_correios/chromedriver")


CRAWLER PROTOCOLOS - login
CRAWLER PROTOCOLOS - login ok
CRAWLER PROTOCOLOS - Iniciando crawler chat
1 - protocolo: 64482585 -  Normal
2 - protocolo: 64482526 -  Normal
3 - protocolo: 64482515 -  Normal
4 - protocolo: 64482503 -  Normal
5 - protocolo: 64482496 -  Normal
6 - protocolo: 64482478 -  Normal
Erro no try - Recarregando pagina
CRAWLER PROTOCOLOS - login
Recarregando pagina
7 - protocolo: 64482473 -  Normal
8 - protocolo: 64482466 -  Normal
Erro no try - Recarregando pagina
CRAWLER PROTOCOLOS - login
Recarregando pagina
9 - protocolo: 64482452 -  Normal
10 - protocolo: 64482404 -  Normal
CRAWLER PROTOCOLOS - Finalizando crawler chat


In [32]:
#Dados Cliente
nome = []
cpf = []
telefone = []
celular = []

for i in dados_comprador:
    a = i.split('\n')
    nome.append(a[1].replace('Nome: ', ''))
    cpf.append(a[2].replace('CPF: ', ''))
    telefone.append(a[3].replace('Telefone fixo: ', '')) 
    celular.append(a[4].replace('Celular: ', '')) 
    
    
df_protocolo['nome'] = nome
df_protocolo['cpf'] = cpf
df_protocolo['telefone'] = telefone
df_protocolo['celular'] = celular

In [60]:
#Dados entrega
dest = []
end = []
bairro = []
cidade = []
uf = []
cep = []

for i in dados_entrega:
    x = i.split('\n')
    dest.append(x[1].replace('Destinatário: ', ''))
    end.append(x[2].replace('Endereço de entrega: ', ''))
    bairro.append(x[3].replace('Bairro: ', '')) 
    cidade.append(x[4].replace('Cidade: ', '')) 
    uf.append(x[5].replace('Estado: ', '')) 
    cep.append(x[6].replace('CEP: ', '')) 
    
    
df_protocolo['destinatario'] = dest
df_protocolo['endereco'] = end
df_protocolo['bairro'] = bairro
df_protocolo['cidade'] = cidade
df_protocolo['uf'] = uf
df_protocolo['cep'] = cep

In [69]:
sku_net = []
sku_seller = []
produto = []
qtd_sku = []
valor = []
data_dev = []

for i in item_troca:
    z = i.split(' ')
    sku_net.append(z[0])
    cod_sku.append(z[1])
    cod_sku.append(z[4])
    cod_sku.append(z[1])
    cod_sku.append(z[1])

    
    
    print(x[0])
    
    


JD8-0571-014-44
JD8-0295-010-05
2IC-9683-006-01
2IC-9642-006-02
2IC-4154-006-04
2IC-9532-030-03
2IC-4776-006-35
2IC-8306-010-03
HZM-2977-010-04
2IC-2782-006-02


In [ ]:
request = RequestSolicitarPostagemReversa(
            codAdministrativo='13415611',
            codigo_servico='04677',
            cartao='0075199017',
            
            destinatario=Destinatario(
                nome='Nike - JP LOGISTICA',
                logradouro='Rodovia Fernão Dias',
                numero='S/N',
                complemento='Km 947.5 G40 Modulo B-NIVEL1',
                bairro='Bairro dos Pires',
                cidade='́Extrema',
                uf='MG',
                cep='37640950'
            ),
    
            coletas_solicitadas=[
                Coleta(
                    tipo='A',
                    valor_declarado=1500.00,
                    remetente=Remetente(
                        nome='Ciclano',
                        logradouro='Rua 35',
                        numero='10',
                        bairro='Águas Claras(Sul)',
                        cidade='Brasília',
                        uf='DF',
                        cep='71931-180',
                        ddd='61',
                        telefone='34262222',
                        email='ciclano@mail.com',
                        identificacao='',
                        ddd_celular='61',
                        celular='92236666',
                        sms='N'
                    ),

                    obj_col=[
                        Objeto(
                            item=1,
                            id='582298235'

                        )
                    ]
                )
            ]
        )

response = cliente.SolicitarPostagemReversa(request)
print(response)


In [ ]:
# ###############################################################################################################
# from unittest import TestCase
# from correios_lib.webservices import LogisticaReversa
# from correios_lib.entities import Remetente, Destinatario, Coleta, \
#      Objeto, Produto, ColetaSimultanea
# from correios_lib.requests import RequestSolicitarPostagemReversa, \
#      RequestAcompanharPedido, RequestAcompanharPedidoPorData, \
#      RequestCancelarPedido, RequestSolicitarRange, \
#      RequestCalcularDigitoVerificador, RequestSolicitarPostagemSimultanea
# import datetime


# # cliente = LogisticaReversa(
# #             env='HOMOLOG',
# #             id_correios='empresacws',
# #             password='123456',
# #             cert=False
# #         )


# request = RequestAcompanharPedidoPorData(
#     codAdministrativo='13415611',
#     tipoSolicitacao='A',
#     data=datetime.date(2022, 11, 14)
# )
# response = cliente.AcompanharPedidoPorData(request)
# print(response)